In [1]:
from elasticsearch import Elasticsearch
es = Elasticsearch('http://localhost:9200/')
es.ping()

True

In [ ]:
import os
API_KEY = os.environ['GEMINI_API_KEY']

In [2]:
import google.generativeai as genai
genai.configure(api_key=API_KEY)

c:\Users\sadhanas\OneDrive - Synopsys, Inc\Documents\Project\ReviewResto\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def create_embedding(text):
    result = genai.embed_content(model="models/text-embedding-004", content=text)
    return result['embedding']

In [20]:
def get_context(query):
    vector = create_embedding(query)
    knn = {
        'field': "review_vector",
        'query_vector': vector,
        'k':50
    }
    res = es.search(index='review_index', knn=knn, source=['restaurant_name','review'])
    documents = []
    for doc in res['hits']['hits']:
        documents.append(str(doc['_source']))
    return documents

def get_suggestions(query):
    model = genai.GenerativeModel("gemini-1.5-flash")
    documents = get_context(query)
    output_format = "json format(recommendations: {restaurant_name, review, note}. conclusion). the output should contain top recommendations with review text and things to be noted(if there is any negative review). at last a conclusion. note: donot recommend a restaurant if there is negative opinion"
    prompt = f"You are a resturant suggestor. suggest resturants based on the given data'{query}'.\n\data:\n{''.join(documents)}. output format: {output_format}"
    response = model.generate_content(prompt)
    return response.text

<>:18: SyntaxWarning: invalid escape sequence '\d'
<>:18: SyntaxWarning: invalid escape sequence '\d'
C:\Users\sadhanas\AppData\Local\Temp\ipykernel_22684\2862575062.py:18: SyntaxWarning: invalid escape sequence '\d'
  prompt = f"You are a resturant suggestor. suggest resturants based on the given data'{query}'.\n\data:\n{''.join(documents)}. output format: {output_format}"


In [23]:
query = "suggest good pizza place"
print(get_suggestions(query))

```json
{
  "recommendations": [
    {
      "restaurant_name": "Kafros Pizzeria",
      "review": "Amazing pizzas omg!",
      "note": "A little expensive according to one review.  Service can be unfriendly according to another review.  Consider these points before deciding."
    },
    {
      "restaurant_name": "Pizza Fellas",
      "review": "They make delicious pizza with nice ingredients on thin dough. Slice pizza and all pizza options are available. You can try two different flavors as a whole by making two different choices for the whole pizza. We wanted the mushroom and sausage. It was very nice; warm; and it was cooked in its consistency without drying the ingredients.",
      "note": "No negative notes mentioned in the provided review."
    },
    {
      "restaurant_name": "Leman Kultur",
      "review": "Location is nice. We love it as a family. Pizza was delicious.",
      "note": "No negative notes mentioned in the provided review."
    },
    {
      "restaurant_name": 

In [13]:
result = get_suggestions(query)
res = result.split('```json')[1].split('```')[0]
import json
result = json.loads(res)
for i in result['recommendations']:
    print(i['restaurant_name'])

J Burber
Zula


In [7]:
def get_res_reviews(name):
    query = {
        "size":1000,
    "query":{
        "match":{
            "restaurant_name":name
            }
        }
    }
    res = es.search(index="reviews", body=query,source=['restaurant_name','review'])
    documents = []
    for doc in res['hits']['hits']:
        documents.append(str(doc['_source']))
    return documents

In [8]:
def get_summary(name):
    model = genai.GenerativeModel("gemini-1.5-flash")
    documents = get_res_reviews(name)
    output_format = "output in json format. the output should contain best dishes to try(if have data). highlights(if data available). points to keep in mind(if have negative data). at last overall conclusion and overall rating"
    prompt = f"You are a resturant suggestor. you provide summary of the given restaurant:'{name}' based on given reviews.\n\reviews:\n{''.join(documents)}. output format{output_format}"
    response = model.generate_content(prompt)
    return response.text

In [1]:
print(get_summary('J Burber'))

NameError: name 'get_summary' is not defined

In [9]:
def restaurant_qna(name, query):
    model = genai.GenerativeModel("gemini-1.5-flash")
    documents = get_res_reviews(name)
    output_format = "json contains restaurant name and answer."
    prompt = f"You are a resturant suggestor.Answer the user's query:'{query} on a restaurant':'{name}' based on given reviews.\n\reviews:\n{''.join(documents)}. output format: {output_format}"
    response = model.generate_content(prompt)
    return response.text

In [10]:
query = 'what about the ambience'
print(restaurant_qna('Kafros Pizzeria',query))

C:\Users\sadhanas\AppData\Local\Temp\ipykernel_9288\3514229138.py:10: DeprecationWarning: Received 'source' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  res = es.search(index="reviews", body=query,source=['restaurant_name','review'])


```json
{
  "restaurant_name": "Kafros Pizzeria",
  "answer": "Reviews describe Kafros Pizzeria as having a 'nice' atmosphere, though one review mentions flies being an issue in the open-air seating.  Several reviews mention it being crowded, sometimes with long wait times, which might impact the overall ambience.  While the overall ambiance is generally considered pleasant, the level of attentiveness and friendliness of the staff varies according to customer experience."
}
```



In [7]:
def general_qna(query):
    model = genai.GenerativeModel("gemini-1.5-flash")
    output_format = "json contains answer."
    prompt = f"Answer the user's general query:'{query} output in the provided format. output format: {output_format}"
    response = model.generate_content(prompt)
    return response.text

print(general_qna("what is Sardines"))

```json
{
  "answer": "Sardines are small, oily fish, typically canned or packed in oil or brine.  Their output depends on the context.  If you mean the fishing industry's output, it's measured in tons caught annually. If you mean the canned goods industry's output, it's measured in the number of cans or total weight of sardines produced.  There's no single, definitive answer without specifying which 'output' you're asking about and in what region or timeframe."
}
```

